# Lab 3 — dimensionality reduction

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

In [ ]:
!hdfs dfs -cat /ix/ml-20m/genome-tags.txt | tail -n 2

In [ ]:
tags = sc.textFile("/ix/ml-20m/genome-tags.txt").map(json.loads)
tag2name = dict(tags.map(itemgetter("tagId", "tag")).collect())
movies = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads)
scores = sc.textFile("/ix/ml-20m/genome-scores.txt").map(json.loads)

movieId_withTags = scores.map(itemgetter('movieId')).distinct()

print(tags.take(1))
print(movies.take(1))
print(scores.take(1))

nb_tags = tags.count()
nb_movies = movies.count()
nb_movieId_withTags = movieId_withTags.count()
nb_scores = scores.count()

print('number tags: ', nb_tags)
print('number movies: ', nb_movies)
print('number movies with tags: ', nb_movieId_withTags)
print('number scores :', nb_scores)

In [ ]:
relevance = scores.map(lambda x: x['relevance'])
matrix = np.array(relevance.collect()).reshape((nb_tags,nb_movieId_withTags), order='F')

In [ ]:
print(matrix)

In [ ]:
tagsVar = np.var(matrix,axis=1)
tagsVarSort = np.sort(tagsVar)

plt.plot(tagsVar)
plt.xlabel('Tag')
plt.ylabel('Variance')
plt.title("Variance of each tag")

In [ ]:
covMatrix = np.cov(matrix)
eigenvalues,eigenvector = np.linalg.eigh(covMatrix)

plt.plot(eigenvaluesCov[0])
plt.title('Eigenvalues of the covariance matrix')
print(np.sum(eigenvalues))
print(np.argmin(eigenvalues[::-1].cumsum() < 2/3*eigenvalues.sum()) )
y = covMatrix.dot(eigenvector)
y = y[-36:,-36:]
print(y)